In [0]:
!pip install tweepy

In [0]:
import tweepy
import pandas as pd

#Add your credentials here
twitter_keys = {
        'consumer_key':        'X',
        'consumer_secret':     'X',
        'access_token_key':    'X',
        'access_token_secret': 'X'
    }

In [0]:
#Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [0]:
tweets = []
from dateutil.parser import parse
from datetime import datetime, timedelta
from pytz import timezone
 
gmt = timezone('Asia/Kolkata')
last_hour_date_time = datetime.now(gmt) - timedelta(hours = 150)

for tweet in tweepy.Cursor(api.search, q="#covid OR #covid19 OR #covid-19 OR #corona OR #StayHome OR #StaySafe OR #coronavirus -filter:retweets", 
                           count=5000,  since=last_hour_date_time.strftime('%Y-%m-%d %H'), tweet_mode='extended', lang="en").items(1000):

#for tweet in tweepy.Cursor(api.search, q="#covid OR #covid19 OR #covid-19 OR #corona OR #StayHome OR #StaySafe OR #coronavirus -filter:retweets", 
#                           count=5000, since='2020-04-06', until='2020-04-07', tweet_mode='extended', lang="en").items(1000):
    data = [tweet.created_at, tweet.full_text, tweet.user._json['name'], tweet.user._json['screen_name'], tweet.user._json['id'], tweet.user._json['location'], 
            tweet.user._json['followers_count'], parse(tweet.user._json['created_at']).strftime("%d-%m-%Y"), tweet.user._json['statuses_count'], tweet.user._json['friends_count'],
            tweet.user._json['listed_count'], tweet.user._json['favourites_count']]

    data = tuple(data)
    tweets.append(data)
    continue
	
df = pd.DataFrame(tweets, columns = ['tweet_time', 'tweet_text', 'username', 'user_screenname', 'userid', 'location', 'followers_count',
                                     'twtr_joined_on', 'statuses_count', 'friends_count', 'listed_count', 'favourites_count'])

print("df ready : ", df.shape )

In [0]:
#Preprocessing
import nltk
nltk.download('punkt')
nltk.download('stopwords')

!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.SMILEY, p.OPT.EMOJI)

In [0]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
cleantweetsNOPunct = []
import string
table = str.maketrans('','', string.punctuation)

for tweet in df['tweet_text']:
    try:
        tweet = p.clean(tweet)
        #tokenize
        words2 = word_tokenize(tweet.lower())
        #remove puncts
        words3 = [w.translate(table) for w in words2]
        cleantweetsNOPunct.append((" ".join(words3)).strip())
    except:
        cleantweetsNOPunct.append(tweet)
        continue

print(len(cleantweetsNOPunct))

df['clean_Tweet_NoPunct'] = cleantweetsNOPunct

In [0]:
#filename = "Covid19_Tweets" + str(datetime.now(gmt).strftime('%Y-%m-%d %H:%M')) + ".csv"
filename = "Covid19_Tweets" + str(last_hour_date_time).replace(" ", "") + ".csv"

df.to_csv(filename, index=False)
print(filename + " stored as csv")

In [0]:
#Code to map extracted Location form Twitter to proper Country Name
!pip install pycountry
import pycountry
import pandas as pd

database = pd.read_csv(filename)
database.shape

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='kunjan6902@yahoo.in')
wordlist = []
countrylist = []

for place in countries:
    try:
        location = geolocator.geocode(place)
        wordlist.extend(str(location).split(','))
        countrylist.append(wordlist[-1])
    except:
        countrylist.append('none')
        continue

database['mapped_country'] = countrylist

df.to_csv("mapped_countries.csv", index=False) 
print("mapped_countries stored as csv")

In [0]:
#Code to translate different country names written in native languages into English
!pip install googletrans
from googletrans import Translator
import pandas as pd

translated = []

database = pd.read_csv("sample_data/mapped_countries.csv")

for cntr_to_translate in database['location']:
    try:
        # REINITIALIZE THE API
        translator = Translator()
        #print(cntr_to_translate)
        done = translator.translate(cntr_to_translate)
        translated.append(done.text)
    except:
        translated.append('none')
        continue

database['trans_country'] = translated

df.to_csv("trnsltd_countries.csv", index=False) 
print("trnsltd_countries stored as csv")

In [0]:
#Code to download Retweets
retweets = []
count = 1

for tweet in tweepy.Cursor(api.search, q="#covid OR #covid19 OR #covid-19 OR #corona OR #StayHome OR #StaySafe OR #coronavirus filter:retweets",
                            count=5000,  since='2020-04-08', tweet_mode='extended', lang="en").items(100),
    data = [tweet.created_at, tweet.full_text, tweet.user._json['name'], tweet.user._json['screen_name'], tweet.user._json['id'], tweet.user._json['location'],
            tweet.user._json['followers_count'], parse(tweet.user._json['created_at']).strftime("%d-%m-%Y"), tweet.user._json['statuses_count'], tweet.user._json['friends_count'],
            tweet.user._json['listed_count'], tweet.user._json['favourites_count']],
    data = tuple(data)
    retweets.append(data)
    except tweepy.TweepError as e:
        print(e.reason)
        continue
    except StopIteration:
        break
        
df = pd.DataFrame(retweets, columns = ['tweet_time', 'tweet_text', 'username', 'user_screenname', 'userid', 'location', 'followers_count',
                                    'twtr_joined_on', 'statuses_count', 'friends_count', 'listed_count', 'favourites_count'])

df.to_csv("Covid19_Retweets_all.csv", index=False),
print("Covid19_Retweets_all stored as csv")